# Arbor YAML Trainer Tutorial with Live Dashboard

This notebook demonstrates how to use the Arbor YAML training system with adaptive context windows and **real-time monitoring dashboard**. Works perfectly in both **local environments** and **Google Colab** with public URL access!

## 🎯 **What You'll Learn:**

1. **Create and customize training configurations**
2. **Train models with dynamic growth and adaptive context**
3. **Monitor training progress with live dashboard visualization**
4. **Track model architecture changes in real-time**
5. **Set up alerts and performance monitoring**
6. **Test the trained model with different task types**

## 🆕 **Dashboard Features:**

- **📊 Live Metrics**: Real-time training loss, learning rate, and gradient monitoring
- **🏗️ Architecture Visualization**: Interactive model structure with layer utilization heatmaps
- **🌱 Growth Tracking**: Timeline of parameter and layer growth events
- **🔔 Alert System**: Automatic notifications for training anomalies and milestones
- **📈 Analytics**: Performance statistics, trends, and exportable reports

## 🌐 **Environment Support:**

### 💻 **Local Development:**
- Dashboard at `http://localhost:8501`
- Full feature access
- Private and secure

### ☁️ **Google Colab:**
- **Public URL** via ngrok (e.g., `https://abc123.ngrok.io`)
- **Mobile-responsive** dashboard
- **Shareable** with team members
- **Identical features** to local setup

## 🚀 **Quick Start:**

### Local:
1. Run this notebook to configure training
2. Start dashboard: `streamlit run arbor/tracking/dashboard.py`
3. Monitor at http://localhost:8501

### Google Colab:
1. Run environment setup cells (auto-detects Colab)
2. Get free ngrok token from [ngrok.com](https://ngrok.com)
3. Run dashboard startup cell with your token
4. Monitor at your unique public URL!

The YAML trainer with Arbor dashboard makes adaptive transformer training accessible everywhere! 🌍

In [ ]:
# Setup and Imports
import os
import sys
import yaml
import torch
from pathlib import Path

# Add arbor to path
sys.path.insert(0, str(Path.cwd()))

# Import Arbor tracking system
try:
    from arbor.tracking import TrainingMonitor
    print("✅ Arbor tracking system imported")
except ImportError as e:
    print(f"⚠️ Arbor tracking not available: {e}")

# Check if we're in the right directory
if not Path("arbor").exists():
    print("❌ Please run this notebook from the arbor-o1-living-ai root directory")
    print("Current directory:", Path.cwd())
else:
    print("✅ Found arbor directory")
    print("📍 Working directory:", Path.cwd())

In [ ]:
# Install Dependencies
print("📦 Installing Required Dependencies...")
print("=" * 50)

# Core dependencies for Arbor training
dependencies = [
    "torch>=2.0.0",
    "transformers>=4.30.0", 
    "datasets>=2.14.0",
    "accelerate>=0.20.0",
    "peft>=0.4.0",
    "bitsandbytes>=0.41.0",
    "streamlit>=1.25.0",
    "plotly>=5.15.0",
    "pyngrok>=6.0.0",  # For Google Colab public URLs
    "tensorboard>=2.13.0",
    "wandb>=0.15.0",  # Optional but useful
    "scipy>=1.10.0",
    "scikit-learn>=1.3.0"
]

# Install other dependencies first
print("\n📦 Installing core dependencies...")
for dep in dependencies:
    try:
        print(f"   Installing {dep}...")
        !pip install -q {dep}
    except Exception as e:
        print(f"   ⚠️ Warning: Could not install {dep}: {e}")

print("\n✅ Core dependencies installed!")

# Now handle Arbor installation with improved logic
print("\n🌱 Installing Arbor package...")
print("   Checking installation options...")

arbor_installed = False

# Option 1: Try GitHub installation (most common failure point)
print("\n🔗 Attempting GitHub installation...")
try:
    !pip install -q git+https://github.com/Noema-Research/Arbor.git
    print("✅ Arbor installed successfully from GitHub!")
    arbor_installed = True
except Exception as e:
    print(f"⚠️ GitHub installation failed: {e}")
    print("   This is common in cloud environments due to access restrictions")

# Option 2: Clone repo and install locally (for Colab)
if not arbor_installed:
    print("\n📥 Trying to clone repository for local installation...")
    try:
        # Check if we're in Colab and need to clone
        import os
        if not os.path.exists("arbor-o1-living-ai"):
            print("   Cloning Arbor repository...")
            !git clone https://github.com/Noema-Research/Arbor.git arbor-o1-living-ai
        
        # Change to the repo directory and install
        if os.path.exists("arbor-o1-living-ai"):
            print("   Installing from cloned repository...")
            !cd arbor-o1-living-ai && pip install -q -e .
            print("✅ Arbor installed from cloned repository!")
            arbor_installed = True
        
    except Exception as e:
        print(f"⚠️ Repository clone/install failed: {e}")

# Option 3: Check if we're already in the Arbor directory
if not arbor_installed:
    print("\n📁 Checking for local Arbor directory...")
    if Path("arbor").exists() and Path("pyproject.toml").exists():
        print("   Found Arbor source - installing in development mode...")
        try:
            !pip install -q -e .
            print("✅ Arbor installed in development mode!")
            arbor_installed = True
        except Exception as e:
            print(f"   ❌ Development installation failed: {e}")

# Option 4: Fallback - manual path setup
if not arbor_installed:
    print("\n🔧 Setting up manual path-based import...")
    import sys
    
    # Try multiple possible paths
    possible_paths = [
        ".",
        "arbor-o1-living-ai", 
        "/content/arbor-o1-living-ai",
        str(Path.cwd())
    ]
    
    for path in possible_paths:
        if Path(path, "arbor").exists():
            sys.path.insert(0, str(Path(path).absolute()))
            print(f"   Added {path} to Python path")
            break
    
    print("   📝 Note: Using path-based import - some features may be limited")

# Verify installations
print("\n🔍 Verifying installations...")

# Check key packages
key_packages = {
    'torch': 'PyTorch',
    'transformers': 'HuggingFace Transformers', 
    'datasets': 'HuggingFace Datasets',
    'streamlit': 'Streamlit Dashboard',
    'plotly': 'Plotly Visualization'
}

for package, name in key_packages.items():
    try:
        __import__(package)
        print(f"   ✅ {name}")
    except ImportError:
        print(f"   ❌ {name} - installation may have failed")

# Special comprehensive check for Arbor
print(f"\n🌱 Checking Arbor installation...")
arbor_available = False

try:
    # First try direct import
    import arbor
    print(f"   ✅ Arbor imported successfully (version: {getattr(arbor, '__version__', 'development')})")
    arbor_available = True
except ImportError as e1:
    print(f"   ⚠️ Direct import failed: {e1}")
    
    # Try with path manipulation
    import sys
    paths_to_try = [".", "arbor-o1-living-ai", "/content/arbor-o1-living-ai"]
    
    for path in paths_to_try:
        try:
            if Path(path, "arbor").exists():
                if str(Path(path).absolute()) not in sys.path:
                    sys.path.insert(0, str(Path(path).absolute()))
                import arbor
                print(f"   ✅ Arbor imported with path: {path}")
                arbor_available = True
                break
        except ImportError:
            continue
    
    if not arbor_available:
        print(f"   ❌ Arbor not available - notebook will work with limited functionality")

print("\n🎯 Installation Summary:")
if arbor_available:
    print("   • ✅ All systems ready - full functionality available!")
    print("   • ✅ Arbor package properly installed and importable")
else:
    print("   • ⚠️ Arbor not fully installed - some features may be limited")
    print("   • 📝 Tutorial will continue with simulations where needed")

print("   • If you see any ❌ errors, try restarting runtime and re-running")
print("   • For Google Colab: Some packages may require runtime restart")
print("   • Cloud environments sometimes have installation restrictions")

print("\n🚀 Ready to proceed with YAML training!")

# Environment-specific tips
try:
    import google.colab
    print("\n💡 Google Colab Tips:")
    print("   • Repository cloned to: /content/arbor-o1-living-ai")
    print("   • Use 'Runtime → Restart runtime' if you encounter import issues")
    print("   • Some packages may require runtime restart to work properly")
    print("   • The notebook is designed to work even with limited Arbor functionality")
    
    # Show current working directory
    import os
    print(f"   • Current directory: {os.getcwd()}")
    if os.path.exists("arbor-o1-living-ai"):
        print("   • ✅ Arbor repository is available")
    else:
        print("   • ⚠️ Arbor repository not found - some features may be limited")
        
except ImportError:
    print("\n💻 Local Environment:")
    print("   • Make sure you're running from the arbor-o1-living-ai directory")
    print("   • Or the notebook will clone the repository automatically")

In [ ]:
# Environment Detection and Colab Setup
import os
import sys

# Detect if we're running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("🌐 Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("💻 Running locally")

# Setup for Google Colab
if IN_COLAB:
    print("🔧 Setting up Google Colab environment...")
    
    # Install additional packages needed for Colab
    print("📦 Installing required packages...")
    !pip install -q streamlit pyngrok plotly
    
    # Setup ngrok for public URL access
    print("🌐 Setting up public URL access with ngrok...")
    
    # Import ngrok after installation
    from pyngrok import ngrok
    import getpass
    
    # Get ngrok auth token (user needs to provide this)
    print("🔑 To access the dashboard publicly, you need an ngrok auth token:")
    print("   1. Go to https://ngrok.com/ and sign up (free)")
    print("   2. Get your auth token from https://dashboard.ngrok.com/get-started/your-authtoken")
    print("   3. Enter it below (or set NGROK_AUTH_TOKEN environment variable)")
    
    ngrok_token = os.getenv('NGROK_AUTH_TOKEN')
    if not ngrok_token:
        print("💡 Tip: Set NGROK_AUTH_TOKEN environment variable to skip this step")
        # For now, we'll handle this when starting the dashboard
        pass
    else:
        ngrok.set_auth_token(ngrok_token)
        print("✅ ngrok authentication configured")
    
    # Set Colab-specific configurations
    DASHBOARD_PORT = 8501
    PUBLIC_URL = None  # Will be set when dashboard starts
    
else:
    # Local development setup
    DASHBOARD_PORT = 8501
    LOCAL_URL = f"http://localhost:{DASHBOARD_PORT}"
    print(f"🏠 Local dashboard will be available at: {LOCAL_URL}")

print("✅ Environment setup complete")

## Step 1: Create Your Training Configuration

The YAML trainer uses configuration files to specify everything about your training run. Let's start by examining and customizing a training configuration.

In [ ]:
# Let's look at the example configuration first
config_path = Path("configs/example_config.yaml")

if config_path.exists():
    with open(config_path, 'r') as f:
        config_content = f.read()
    
    print("📋 Example Configuration Structure:")
    print("=" * 50)
    
    # Show first 30 lines to get an overview
    lines = config_content.split('\n')
    for i, line in enumerate(lines[:30]):
        print(f"{i+1:2d}: {line}")
    
    if len(lines) > 30:
        print(f"... ({len(lines) - 30} more lines)")
else:
    print("❌ Example config not found. Let's create one!")

In [ ]:
# Create optimized T4 GPU configuration for 1B model with all features
print("🚀 Creating T4-Optimized 1B Model Configuration")
print("=" * 60)

t4_optimized_config = {
    'model': {
        'name': 'arbor-1b-t4-demo',
        'vocab_size': 128256,
        
        # 1B parameter architecture optimized for T4
        'hidden_size': 2048,      # Good balance for T4 memory
        'num_layers': 24,         # Starting layers (will grow to 32)
        'num_heads': 16,          # 2048 / 16 = 128 head dim
        'ffn_dim': 5504,          # ~2.7x hidden_size for efficiency
        'max_seq_length': 4096,   # Good for most tasks
        
        # Layer Growth Configuration
        'layer_growth': {
            'enabled': True,
            'min_layers': 24,
            'max_layers': 32,           # Conservative for T4 memory
            'growth_factor': 2,         # Add 2 layers at a time
            'growth_threshold': 0.88,   # Trigger when 88% utilized
            'growth_cooldown': 100,     # Wait 100 steps between growth
            'utilization_window': 10    # Average over 10 steps
        },
        
        # Parameter Growth Configuration  
        'growth': {
            'enabled': True,
            'factor': 1.5,              # Moderate growth for stability
            'threshold': 0.85,          # Conservative threshold
            'max_steps': 3,             # Max 3 growth events
            'cooldown': 75              # Wait between parameter growth
        },
        
        # Adaptive Context (great for demos)
        'adaptive_context': {
            'enabled': True,
            'min_context_length': 512,
            'max_context_length': 4096,
            'context_lengths': [512, 1024, 2048, 4096],
            'task_types': ['chat', 'code', 'reasoning', 'document'],
            'adaptation_threshold': 0.7
        },
        
        # T4 Memory Optimizations
        'use_cache': False,           # Save memory during training
        'gradient_checkpointing': True,
        'attention_dropout': 0.1,
        'hidden_dropout': 0.1
    },
    
    # Required tokenizer section
    'tokenizer': {
        'name': 'NousResearch/Hermes-3-Llama-3.1-8B',
        'model_max_length': 4096,
        'padding_side': 'left',
        'truncation_side': 'right',
        'add_eos_token': True,
        'add_bos_token': True,
        'use_fast': True,
        'trust_remote_code': False
    },
    
    'datasets': [
        {
            'name': 'tinystories_demo',
            'source': 'roneneldan/TinyStories',
            'split': 'train[:5000]',    # Small for demo
            'text_column': 'text',
            'preprocessing': {
                'prefix': '📚 Story: ',
                'suffix': ' [END]',
                'max_length': 1024,     # Shorter for variety
                'min_length': 100
            }
        },
        {
            'name': 'code_demo',
            'source': 'codeparrot/github-code-clean',
            'split': 'train[:2000]',    # Small code samples
            'text_column': 'code',
            'preprocessing': {
                'prefix': '💻 Code:\n',
                'suffix': '\n# End of code',
                'max_length': 2048,     # Longer for code
                'filter_languages': ['python', 'javascript']
            }
        },
        {
            'name': 'reasoning_demo',
            'source': 'microsoft/orca-math-word-problems-200k',
            'split': 'train[:1000]',    # Math reasoning
            'text_column': 'question',
            'preprocessing': {
                'prefix': '🧠 Problem: ',
                'suffix': ' [SOLVE]',
                'max_length': 1536
            }
        }
    ],
    
    'training': {
        'output_dir': './arbor-1b-t4-demo',
        
        # T4-Optimized Training Settings
        'learning_rate': 2e-4,            # Good starting LR for 1B
        'warmup_steps': 200,              # Gradual warmup
        'steps_per_dataset': 500,         # 1500 total steps across 3 datasets
        'max_train_steps': 1500,
        
        # T4 Memory Management
        'per_device_train_batch_size': 1,  # Small batch for T4
        'gradient_accumulation_steps': 16, # Effective batch size = 16
        'dataloader_num_workers': 2,       # Don't overwhelm T4
        
        # Mixed Precision & Optimization
        'fp16': True,                      # Essential for T4
        'bf16': False,                     # T4 doesn't support bf16
        'gradient_checkpointing': True,    # Save memory
        'optim': 'adamw_torch',
        'weight_decay': 0.01,
        'adam_beta1': 0.9,
        'adam_beta2': 0.999,
        'adam_epsilon': 1e-8,
        'max_grad_norm': 1.0,
        
        # Evaluation & Saving
        'eval_steps': 100,                 # Frequent evaluation
        'save_steps': 250,                 # Save checkpoints
        'save_total_limit': 3,
        'logging_steps': 25,               # Detailed logging
        
        # Efficiency Settings
        'remove_unused_columns': True,
        'prediction_loss_only': True,
        'use_mps_device': False           # T4 is CUDA
    },
    
    'logging': {
        'arbor_tracking': {
            'enabled': True,
            'save_dir': './t4_training_logs',
            'update_interval': 0.5,        # Frequent updates for demo
            'dashboard_port': 8502,        # Different port for demo
            'alerts': {
                'enabled': True,
                'email_notifications': False,
                'webhook_url': None,
                'alert_thresholds': {
                    'loss_spike': 2.0,     # Alert if loss > 2x avg
                    'memory_usage': 0.9,   # Alert at 90% memory
                    'gradient_norm': 10.0, # Alert for gradient explosion
                    'learning_rate_drop': 0.5  # Alert for LR scheduler issues
                }
            },
            'metrics': {
                'track_gpu_memory': True,
                'track_layer_utilization': True,
                'track_gradient_norms': True,
                'track_learning_rates': True,
                'export_frequency': 'every_100_steps'
            }
        },
        'console': {
            'enabled': True,
            'level': 'INFO',
            'format': '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
        },
        'tensorboard': {
            'enabled': True,
            'log_dir': './t4_tensorboard_logs'
        }
    },
    
    'huggingface': {
        'upload': {
            'enabled': False,              # Can enable for demo
            'repository': 'your-username/arbor-1b-t4-demo',
            'private': False,
            'push_to_hub_frequency': 500
        },
        'token': None  # Set your HF token if uploading
    },
    
    # Demo-specific features
    'demo_features': {
        'enable_growth_visualization': True,
        'save_architecture_snapshots': True,
        'generate_sample_outputs': True,
        'create_training_video': False,    # Set True for video generation
        'benchmark_performance': True
    }
}

# Save the T4-optimized configuration
t4_config_path = Path("configs/t4_1b_demo_config.yaml")
t4_config_path.parent.mkdir(exist_ok=True)

with open(t4_config_path, 'w') as f:
    yaml.dump(t4_optimized_config, f, default_flow_style=False, indent=2)

print("✅ Created T4-optimized 1B model configuration!")
print(f"📁 Saved to: {t4_config_path}")
print("\n🎯 Configuration Highlights:")
print("=" * 50)
print(f"📊 Model: ~1B parameters (2048 hidden, 24→32 layers)")
print(f"🧠 Memory: Optimized for T4 GPU (16GB)")
print(f"🌱 Growth: Layer growth (24→32) + parameter growth")
print(f"📈 Dashboard: Enhanced tracking at http://localhost:8502")
print(f"⚡ Training: 1500 steps across 3 diverse datasets")
print(f"🔄 Context: Adaptive 512→4096 tokens")
print(f"💾 Batch: Effective size 16 (1×16 accumulation)")
print(f"🎯 Features: All demo features enabled")

print("\n💡 T4 Optimizations:")
print("  • FP16 precision for memory efficiency")
print("  • Gradient checkpointing enabled")
print("  • Small batch size with accumulation")
print("  • Conservative growth thresholds")
print("  • Frequent logging for monitoring")

print("\n🚀 Perfect for:")
print("  • Live training demonstrations")
print("  • Growth visualization")
print("  • Real-time dashboard monitoring")
print("  • T4/Colab compatibility")
print("  • Educational showcases")

print("\n✅ Configuration Structure:")
print("  • ✅ model section - architecture and growth settings")
print("  • ✅ tokenizer section - Hermes-3-Llama-3.1-8B configuration")
print("  • ✅ datasets section - 3 diverse training datasets")
print("  • ✅ training section - T4-optimized training parameters")
print("  • ✅ logging section - comprehensive monitoring setup")
print("  • ✅ All required sections for YAML trainer")

In [ ]:
# How to use the T4-optimized configuration
print("🎮 How to Use the T4 Configuration")
print("=" * 50)

print("1. 🚀 Quick Start (Local with T4):")
print("   python train.py configs/t4_1b_demo_config.yaml")
print()

print("2. 📊 With Dashboard (Recommended):")
print("   # Terminal 1: Start dashboard")
print("   streamlit run arbor/tracking/dashboard.py --server.port=8502")
print("   # Terminal 2: Start training")  
print("   python train.py configs/t4_1b_demo_config.yaml")
print("   # View at: http://localhost:8502")
print()

print("3. 🌐 Google Colab with T4:")
print("   # Enable T4 GPU: Runtime → Change runtime type → T4 GPU")
print("   # Run the Colab setup cells above")
print("   # Use this config with the dashboard!")
print()

print("4. 📈 What You'll See:")
print("   • Real-time loss curves across 3 different datasets")
print("   • Layer growth from 24 → 32 layers during training")
print("   • Parameter growth events (FFN expansion)")
print("   • Adaptive context switching between task types")
print("   • GPU memory utilization tracking")
print("   • Growth event alerts and notifications")
print()

print("5. 💾 Expected Training Time:")
print("   • T4 GPU: ~45-60 minutes for full 1500 steps")
print("   • Checkpoints saved every 250 steps")
print("   • Dashboard updates every 0.5 seconds")
print("   • Growth events typically occur around steps 400, 800, 1200")
print()

print("6. 🎯 Demo Scenarios:")
print("   • Show adaptive context: Watch context length change by dataset")
print("   • Demonstrate growth: Model architecture evolves during training")
print("   • Monitor performance: Real-time GPU utilization and efficiency")
print("   • Alert system: Get notified of training events")
print()

# Memory estimation
print("💾 Memory Requirements:")
print("=" * 25)
model_params = 1.0  # 1B parameters
bytes_per_param = 4  # FP32
fp16_savings = 0.5   # FP16 uses half memory
gradient_multiplier = 2  # Gradients + optimizer states
activation_memory = 2    # Activation memory (GB)

base_memory = model_params * bytes_per_param * fp16_savings * gradient_multiplier
total_memory = base_memory + activation_memory

print(f"Model parameters: {model_params:.1f}B")
print(f"Base memory (FP16): {base_memory:.1f}GB")
print(f"Activation memory: {activation_memory:.1f}GB")
print(f"Total estimated: {total_memory:.1f}GB")
print(f"T4 GPU memory: 16GB")
print(f"Memory utilization: {(total_memory/16)*100:.1f}%")
print("✅ Fits comfortably on T4!")

print(f"\n🔧 To customize this config:")
print(f"   • Edit configs/t4_1b_demo_config.yaml")
print(f"   • Adjust batch_size/accumulation for your GPU")
print(f"   • Change growth thresholds for more/less growth")
print(f"   • Add your own datasets")
print(f"   • Configure alerts and notifications")

# Set this as the active tutorial config
tutorial_config = t4_optimized_config
tutorial_config_path = t4_config_path

print(f"\n✅ T4 config is now active for the rest of this tutorial!")

## Step 2: Understanding the YAML Configuration

Let's examine the key sections of our configuration and what they control:

In [ ]:
# Let's explore each section of our configuration
print("🔍 Configuration Analysis")
print("=" * 50)

# Model configuration
model_config = tutorial_config['model']
print("\n🤖 MODEL CONFIGURATION:")
print(f"   Vocabulary: {model_config['vocab_size']:,} tokens (Hermes-4-405B)")
print(f"   Architecture: {model_config['num_layers']} layers × {model_config['hidden_size']} dim")
print(f"   Parameters: ~{(model_config['hidden_size'] * model_config['num_layers'] * 4) / 1e6:.0f}M")

# Growth settings
growth = model_config['growth']
print(f"\n🌱 GROWTH SETTINGS:")
print(f"   Enabled: {growth['enabled']}")
print(f"   Growth factor: {growth['factor']}x")
print(f"   Max growth steps: {growth['max_steps']}")
print(f"   Trigger threshold: {growth['threshold']}")

# Adaptive context
adaptive = model_config['adaptive_context']
print(f"\n🧠 ADAPTIVE CONTEXT:")
print(f"   Enabled: {adaptive['enabled']}")
print(f"   Context range: {adaptive['min_context_length']:,} - {adaptive['max_context_length']:,}")
print(f"   Task types: {len(adaptive['task_types'])} ({', '.join(adaptive['task_types'])})")
print(f"   Context options: {len(adaptive['context_lengths'])} levels")

# Datasets
datasets = tutorial_config['datasets']
print(f"\n📚 DATASETS:")
for i, dataset in enumerate(datasets, 1):
    print(f"   {i}. {dataset['name']}: {dataset['source']} ({dataset['split']})")
    print(f"      Max length: {dataset['preprocessing']['max_length']} tokens")

# Tracking and logging
logging_config = tutorial_config['logging']
print(f"\n📊 ARBOR TRACKING & LOGGING:")
arbor_tracking = logging_config['arbor_tracking']
print(f"   Dashboard enabled: {arbor_tracking['enabled']}")
print(f"   Save directory: {arbor_tracking['save_dir']}")
print(f"   Dashboard port: {arbor_tracking['dashboard_port']}")
print(f"   Live monitoring: {arbor_tracking['update_interval']}s intervals")
print(f"   Alerts enabled: {arbor_tracking['alerts']['enabled']}")
print(f"   Console logging: {logging_config['console']['enabled']} ({logging_config['console']['level']})")

# Training
training = tutorial_config['training']
print(f"\n🎯 TRAINING:")
print(f"   Learning rate: {training['learning_rate']}")
print(f"   Batch size: {training['per_device_train_batch_size']} × {training['gradient_accumulation_steps']} = {training['per_device_train_batch_size'] * training['gradient_accumulation_steps']}")
print(f"   Total steps: {len(datasets)} × {training['steps_per_dataset']} = {len(datasets) * training['steps_per_dataset']}")
print(f"   Mixed precision: {training['fp16']}")

## Step 3: Initialize the YAML Trainer

Now let's create and initialize the YAML trainer with our configuration:

In [ ]:
# Import the YAML trainer with robust error handling
print("📥 Importing YAML Trainer...")
print("=" * 50)

# First, ensure we have the necessary imports
import sys
from pathlib import Path

# Try to import the YAML trainer with fallbacks
trainer_imported = False
import_error = None

try:
    from arbor.train.yaml_trainer import ArborYAMLTrainer
    print("✅ Successfully imported ArborYAMLTrainer")
    trainer_imported = True
except ImportError as e:
    import_error = e
    print(f"❌ Import error: {e}")
    
    # Try adding paths and importing again
    possible_paths = [".", "arbor-o1-living-ai", "/content/arbor-o1-living-ai"]
    
    for path in possible_paths:
        if Path(path, "arbor").exists():
            print(f"   Trying path: {path}")
            if str(Path(path).absolute()) not in sys.path:
                sys.path.insert(0, str(Path(path).absolute()))
            try:
                from arbor.train.yaml_trainer import ArborYAMLTrainer
                print(f"✅ Successfully imported ArborYAMLTrainer from {path}")
                trainer_imported = True
                break
            except ImportError as e2:
                print(f"   Failed from {path}: {e2}")
                continue

if not trainer_imported:
    print("\n🔧 Trainer import failed - trying alternative approaches...")
    
    # Check if we're in Colab and need to install/setup
    try:
        import google.colab
        print("🌐 Detected Google Colab environment")
        print("💡 Suggestions:")
        print("   1. Make sure you ran the dependencies installation cell successfully")
        print("   2. Try restarting the runtime: Runtime → Restart runtime")
        print("   3. Re-run all cells from the beginning")
        print("   4. Check that Arbor was installed properly")
    except ImportError:
        print("💻 Local environment detected")
        print("💡 Suggestions:")
        print("   1. Make sure you're in the arbor-o1-living-ai directory")
        print("   2. Try: pip install -e .")
        print("   3. Check that all dependencies are installed")
    
    print(f"\n❌ Could not import ArborYAMLTrainer")
    print(f"   Original error: {import_error}")
    print("   The notebook will continue with limited functionality")
    
    # Create a mock trainer class for demonstration
    class MockArborYAMLTrainer:
        def __init__(self, config_path):
            self.config_path = config_path
            print("🔧 Using mock trainer for demonstration")
            
        def setup_tokenizer(self):
            print("🔧 Mock: Would setup tokenizer")
            
        def setup_model(self):
            print("🔧 Mock: Would setup model")
            
        def load_datasets(self):
            print("🔧 Mock: Would load datasets")
            
        def train(self):
            print("🔧 Mock: Would run training")
    
    ArborYAMLTrainer = MockArborYAMLTrainer
    print("✅ Mock trainer class created for demonstration purposes")

else:
    print("\n✅ Trainer import successful - full functionality available!")

print("\n📋 Next steps:")
if trainer_imported:
    print("   • Ready to initialize trainer with configuration")
    print("   • All Arbor features will be available")
else:
    print("   • Will use mock trainer for demonstration")
    print("   • Fix import issues for full functionality")
    print("   • Check installation and restart runtime if needed")

In [ ]:
# Initialize the trainer with robust error handling
print("🚀 Initializing YAML Trainer...")
print("=" * 50)

# Check if we have the required variables
config_available = False
config_path_to_use = None

# Check for tutorial config path
if 'tutorial_config_path' in globals():
    config_path_to_use = tutorial_config_path
    config_available = True
    print(f"✅ Found tutorial_config_path: {tutorial_config_path}")
elif 'tutorial_config' in globals():
    # If we have the config dict but not the path, create a temporary config file
    import tempfile
    import yaml
    
    temp_config = tempfile.NamedTemporaryFile(mode='w', suffix='.yaml', delete=False)
    yaml.dump(tutorial_config, temp_config, default_flow_style=False)
    temp_config.close()
    
    config_path_to_use = Path(temp_config.name)
    config_available = True
    print(f"✅ Created temporary config file: {config_path_to_use}")
else:
    # Create a minimal config for demonstration with all required sections
    print("⚠️ No tutorial config found - creating minimal config for demonstration")
    
    minimal_config = {
        'model': {
            'name': 'demo-model',
            'vocab_size': 50257,
            'hidden_size': 512,
            'num_layers': 6,
            'num_heads': 8,
            'max_seq_length': 1024,
            'growth': {
                'enabled': False
            },
            'adaptive_context': {
                'enabled': False
            }
        },
        # Required tokenizer section
        'tokenizer': {
            'name': 'gpt2',
            'model_max_length': 1024,
            'padding_side': 'left',
            'add_eos_token': True,
            'use_fast': True
        },
        'datasets': [{
            'name': 'demo_dataset',
            'source': 'roneneldan/TinyStories',
            'split': 'train[:100]',
            'text_column': 'text',
            'preprocessing': {
                'max_length': 512
            }
        }],
        'training': {
            'output_dir': './demo_output',
            'learning_rate': 5e-5,
            'per_device_train_batch_size': 1,
            'max_train_steps': 10,
            'logging_steps': 5,
            'save_steps': 10
        },
        'logging': {
            'console': {
                'enabled': True,
                'level': 'INFO'
            }
        }
    }
    
    import tempfile
    import yaml
    
    temp_config = tempfile.NamedTemporaryFile(mode='w', suffix='.yaml', delete=False)
    yaml.dump(minimal_config, temp_config, default_flow_style=False)
    temp_config.close()
    
    config_path_to_use = Path(temp_config.name)
    config_available = True
    print(f"✅ Created minimal demo config: {config_path_to_use}")
    print("   📋 Includes all required sections: model, tokenizer, datasets, training")

# Now try to initialize the trainer
trainer = None
trainer_initialized = False

if config_available and 'ArborYAMLTrainer' in globals():
    try:
        trainer = ArborYAMLTrainer(str(config_path_to_use))
        print("✅ Trainer initialized successfully!")
        trainer_initialized = True
        
        # The trainer automatically validates the configuration
        print("📋 Configuration loaded and validated")
        
        # Show config info if available
        if hasattr(trainer, 'config'):
            print(f"📊 Config loaded from: {config_path_to_use}")
            
            # Show which sections were loaded
            config_sections = []
            if hasattr(trainer.config, 'model_config'):
                config_sections.append("model")
            if hasattr(trainer.config, 'tokenizer_config'):
                config_sections.append("tokenizer")
            if hasattr(trainer.config, 'dataset_config'):
                config_sections.append("datasets")
            if hasattr(trainer.config, 'training_config'):
                config_sections.append("training")
            
            print(f"📋 Loaded sections: {', '.join(config_sections)}")
        
    except Exception as e:
        print(f"❌ Trainer initialization failed: {e}")
        print("🔧 Creating mock trainer for demonstration...")
        
        # Create a simple mock trainer with basic functionality
        class DemoTrainer:
            def __init__(self, config_path):
                self.config_path = config_path
                self.datasets = {}
                self.model = None
                self.tokenizer = None
                
            def setup_tokenizer(self):
                print("🔧 Demo: Setting up tokenizer...")
                self.tokenizer = MockTokenizer()
                
            def setup_model(self):
                print("🔧 Demo: Setting up model...")
                self.model = MockModel()
                
            def load_datasets(self):
                print("🔧 Demo: Loading datasets...")
                self.datasets = {"demo": MockDataset()}
                
            def train(self):
                print("🔧 Demo: Would run training pipeline...")
        
        class MockTokenizer:
            def encode(self, text):
                return list(range(len(text.split())))
            def decode(self, tokens):
                return " ".join([f"token_{i}" for i in tokens])
            def __len__(self):
                return 50000
        
        class MockModel:
            def param_count(self):
                return 1000000
            def get_context_info(self):
                return {
                    'adaptive_context_enabled': True,
                    'current_context_length': 1024,
                    'min_context_length': 512,
                    'max_context_length': 4096
                }
        
        class MockDataset:
            def __len__(self):
                return 100
            def __getitem__(self, idx):
                return {'input_ids': [1, 2, 3, 4, 5]}
        
        trainer = DemoTrainer(str(config_path_to_use))
        print("✅ Demo trainer created for tutorial purposes")
        trainer_initialized = True

else:
    print("❌ Cannot initialize trainer:")
    if not config_available:
        print("   • No configuration available")
    if 'ArborYAMLTrainer' not in globals():
        print("   • ArborYAMLTrainer not imported")
    
    print("\n💡 To fix this:")
    print("   1. Make sure you ran the dependencies installation cell")
    print("   2. Run the configuration creation cell")
    print("   3. Check that imports worked properly")
    print("   4. Consider restarting runtime and running all cells")

print("\n📊 Trainer Status:")
if trainer_initialized:
    print("   ✅ Trainer ready for use")
    print("   ✅ Can proceed with tutorial steps")
    if hasattr(trainer, 'config'):
        print("   ✅ Configuration validation passed")
else:
    print("   ❌ Trainer not available")
    print("   ⚠️ Tutorial will have limited functionality")

# Make trainer available globally
if trainer:
    globals()['trainer'] = trainer

## Step 4: Setup Components

The trainer needs to setup several components before training. Let's do this step by step to see what's happening:

In [ ]:
# Step 4a: Setup tokenizer
print("📥 Setting up tokenizer...")
try:
    trainer.setup_tokenizer()
    print(f"✅ Tokenizer ready: {len(trainer.tokenizer):,} vocabulary")
    
    # Test the tokenizer
    test_text = "Hello, this is a test of the Hermes tokenizer!"
    tokens = trainer.tokenizer.encode(test_text)
    print(f"🧪 Test encoding: '{test_text}' → {len(tokens)} tokens")
    print(f"   First 10 tokens: {tokens[:10]}")
    
except Exception as e:
    print(f"❌ Tokenizer setup failed: {e}")
    print("This might be due to internet connectivity or HuggingFace access")

In [ ]:
# Step 4b: Setup model
print("\n🤖 Setting up Arbor model...")
try:
    trainer.setup_model()
    print(f"✅ Model created: {trainer.model.param_count():,} parameters")
    
    # Show model architecture details
    config = trainer.model.config
    print(f"\n📊 Model details:")
    print(f"   Architecture: {config.num_layers} layers")
    print(f"   Hidden size: {config.dim}")
    print(f"   Attention heads: {config.num_heads}")
    print(f"   FFN dimension: {config.ffn_dim}")
    print(f"   Max sequence length: {config.max_seq_length:,}")
    
    # Show adaptive context info
    if hasattr(trainer.model, 'get_context_info'):
        context_info = trainer.model.get_context_info()
        print(f"\n🧠 Adaptive context info:")
        print(f"   Enabled: {context_info['adaptive_context_enabled']}")
        if context_info['adaptive_context_enabled']:
            print(f"   Current context: {context_info['current_context_length']:,}")
            print(f"   Context range: {context_info['min_context_length']:,} - {context_info['max_context_length']:,}")
    
except Exception as e:
    print(f"❌ Model setup failed: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Step 4c: Load datasets with robust error handling
print("\n📚 Loading datasets...")

# Check if trainer is available
if 'trainer' not in globals() or trainer is None:
    print("❌ Trainer not available - cannot load datasets")
    print("💡 Please run the trainer initialization cells first")
else:
    try:
        # Check if this is a real trainer or mock trainer
        trainer_type = type(trainer).__name__
        print(f"🔍 Using trainer type: {trainer_type}")
        
        if hasattr(trainer, 'load_datasets') and callable(trainer.load_datasets):
            trainer.load_datasets()
            
            # Check if datasets were loaded successfully
            if hasattr(trainer, 'datasets') and trainer.datasets:
                print(f"✅ Datasets loaded: {len(trainer.datasets)}")
                
                # Show dataset info
                for name, dataset in trainer.datasets.items():
                    print(f"\n📊 Dataset: {name}")
                    print(f"   Size: {len(dataset):,} examples")
                    
                    # Show a sample if possible
                    if len(dataset) > 0:
                        try:
                            sample = dataset[0]
                            if isinstance(sample, dict) and 'input_ids' in sample:
                                input_ids = sample['input_ids']
                                if hasattr(trainer, 'tokenizer') and trainer.tokenizer:
                                    try:
                                        decoded = trainer.tokenizer.decode(input_ids[:50])  # First 50 tokens
                                        print(f"   Sample: {decoded}...")
                                        print(f"   Token length: {len(input_ids)}")
                                    except Exception as e:
                                        print(f"   Sample tokens: {input_ids[:10]}...")
                                        print(f"   Token length: {len(input_ids)}")
                                else:
                                    print(f"   Sample tokens: {input_ids[:10]}...")
                                    print(f"   Token length: {len(input_ids)}")
                            else:
                                print(f"   Sample: {str(sample)[:100]}...")
                        except Exception as e:
                            print(f"   Sample preview failed: {e}")
            else:
                print("⚠️ No datasets found in trainer")
                
        else:
            print("🔧 Mock trainer detected - simulating dataset loading...")
            
            # Create mock datasets for demonstration
            class MockDataset:
                def __init__(self, name, size=1000):
                    self.name = name
                    self.size = size
                    
                def __len__(self):
                    return self.size
                    
                def __getitem__(self, idx):
                    return {
                        'input_ids': list(range(idx, idx + 20)),  # Mock token sequence
                        'attention_mask': [1] * 20
                    }
            
            # Simulate the datasets from the config
            mock_datasets = {
                'tinystories_demo': MockDataset('tinystories_demo', 5000),
                'code_demo': MockDataset('code_demo', 2000),
                'reasoning_demo': MockDataset('reasoning_demo', 1000)
            }
            
            trainer.datasets = mock_datasets
            print("✅ Mock datasets created for demonstration")
            
            for name, dataset in mock_datasets.items():
                print(f"\n📊 Dataset: {name}")
                print(f"   Size: {len(dataset):,} examples")
                print(f"   Sample: Mock data with token sequences")
                print(f"   Token length: 20 (simulated)")
        
    except Exception as e:
        print(f"❌ Dataset loading failed: {e}")
        print("This might be due to:")
        print("   • Internet connectivity issues")
        print("   • HuggingFace dataset access problems")
        print("   • Missing dependencies")
        print("   • Configuration errors")
        
        import traceback
        print(f"\nDetailed error:")
        traceback.print_exc()
        
        print(f"\n🔧 Creating fallback mock datasets...")
        
        # Create simple mock datasets as fallback
        class SimpleDataset:
            def __init__(self, size):
                self.size = size
                
            def __len__(self):
                return self.size
                
            def __getitem__(self, idx):
                return {'input_ids': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
        
        if hasattr(trainer, 'datasets'):
            trainer.datasets = {
                'demo_dataset': SimpleDataset(100)
            }
            print("✅ Fallback datasets created")
        else:
            print("❌ Cannot create fallback datasets - trainer structure unknown")

print("\n📊 Dataset Loading Summary:")
if 'trainer' in globals() and hasattr(trainer, 'datasets') and trainer.datasets:
    print(f"   ✅ {len(trainer.datasets)} datasets available")
    for name in trainer.datasets.keys():
        print(f"   • {name}")
else:
    print("   ⚠️ No datasets available")
    print("   • Check trainer initialization")
    print("   • Verify internet connectivity")
    print("   • Consider using mock data for demonstration")

## Step 5: Training Setup

Before we start training, let's setup logging and create the trainer objects:

In [ ]:
# Setup Arbor tracking and logging
print("📊 Setting up Arbor tracking system...")
try:
    # Initialize TrainingMonitor with configuration
    tracking_config = tutorial_config['logging']['arbor_tracking']
    
    # Create training monitor
    training_monitor = TrainingMonitor(
        save_dir=tracking_config['save_dir'],
        update_interval=tracking_config['update_interval']
    )
    
    # Setup alert system if enabled
    if tracking_config['alerts']['enabled']:
        training_monitor.setup_alerts(
            email_enabled=tracking_config['alerts']['email_notifications'],
            webhook_url=tracking_config['alerts']['webhook_url']
        )
    
    print("✅ Arbor tracking system initialized")
    print(f"   📊 Dashboard will be available at: http://localhost:{tracking_config['dashboard_port']}")
    print(f"   💾 Metrics saved to: {tracking_config['save_dir']}")
    print(f"   🔔 Alerts enabled: {tracking_config['alerts']['enabled']}")
    
    # Setup traditional logging as well
    trainer.setup_logging()
    print("✅ Console logging configured")
    
except Exception as e:
    print(f"⚠️ Tracking setup had issues: {e}")
    print("Training can continue with basic logging")
    # Fallback to basic logging
    try:
        trainer.setup_logging()
        print("✅ Basic logging configured")
    except:
        print("⚠️ Even basic logging had issues - continuing anyway")

In [ ]:
# Start the Arbor Dashboard
print("\n🌐 Starting Arbor Dashboard...")
print("=" * 50)

dashboard_port = tutorial_config['logging']['arbor_tracking']['dashboard_port']

if IN_COLAB:
    print("🌐 Google Colab Setup:")
    print("=" * 30)
    print("To access the dashboard in Colab, we'll create a public URL using ngrok.")
    print("")
    print("📋 Steps to access the dashboard:")
    print("1. 🔑 Get an ngrok token from https://ngrok.com (free signup)")
    print("2. 🚀 Run the dashboard startup cell below")
    print("3. 🌐 Click the public ngrok URL to access the dashboard")
    print("")
    print("💡 The dashboard will show:")
    print("   • 📈 Live training metrics (loss, learning rate)")
    print("   • 🏗️ Model architecture with layer utilization")  
    print("   • 🌱 Growth tracking (parameters and layers)")
    print("   • 🔔 Training alerts and notifications")
    print("   • 📊 Performance analytics and system monitoring")
    print("")
    print("⚠️ Note: In Colab, the dashboard will be accessible via a public ngrok URL")
    print("        Anyone with the URL can access it (ngrok free tier limitation)")
else:
    print("💻 Local Development Setup:")
    print("=" * 30)
    print("To monitor training locally:")
    print("1. 🚀 Option A - Use the launcher:")
    print("   python launch_training_dashboard.py")
    print()
    print("2. 📊 Option B - Start dashboard manually:")
    print(f"   streamlit run arbor/tracking/dashboard.py --server.port={dashboard_port}")
    print()
    print("3. 🌐 Then open your browser to:")
    print(f"   http://localhost:{dashboard_port}")

print()
print("📊 Dashboard Features:")
print("   • 📈 Live training metrics with real-time updates")
print("   • 🏗️ Interactive model architecture visualization")
print("   • 🌱 Parameter and layer growth timeline")
print("   • 🔔 Alert system with configurable notifications")
print("   • 📊 Performance analytics and export capabilities")
print()

if IN_COLAB:
    print("🔧 Next: Run the 'Start Dashboard in Colab' cell below to get your public URL!")
else:
    print("💡 Tip: Start the dashboard in another terminal before running training!")
    print("    The dashboard will automatically pick up training data as it's generated.")

In [ ]:
# Start Dashboard in Colab (with public URL)
if IN_COLAB:
    print("🚀 Starting Arbor Dashboard in Google Colab")
    print("=" * 50)
    
    # Check if ngrok auth token is available
    ngrok_token = os.getenv('NGROK_AUTH_TOKEN')
    if not ngrok_token:
        print("🔑 Please enter your ngrok auth token:")
        print("   Get it from: https://dashboard.ngrok.com/get-started/your-authtoken")
        ngrok_token = getpass.getpass("Enter ngrok auth token: ")
        
        if ngrok_token.strip():
            ngrok.set_auth_token(ngrok_token.strip())
            print("✅ ngrok authentication configured")
        else:
            print("❌ No token provided - dashboard will only be accessible locally")
            print("   You can still run the dashboard, but won't get a public URL")
    
    # Start the dashboard with threading to run in background
    import threading
    import subprocess
    import time
    
    # Create a function to run streamlit
    def run_streamlit():
        try:
            # Change to the correct directory and run streamlit
            dashboard_cmd = [
                sys.executable, "-m", "streamlit", "run", 
                "arbor/tracking/dashboard.py",
                "--server.port", str(DASHBOARD_PORT),
                "--server.headless", "true",
                "--server.enableCORS", "false",
                "--browser.gatherUsageStats", "false"
            ]
            
            print(f"🎬 Starting Streamlit dashboard...")
            subprocess.run(dashboard_cmd)
            
        except Exception as e:
            print(f"❌ Failed to start dashboard: {e}")
    
    # Start streamlit in background thread
    dashboard_thread = threading.Thread(target=run_streamlit, daemon=True)
    dashboard_thread.start()
    
    # Wait a moment for streamlit to start
    print("⏳ Waiting for dashboard to initialize...")
    time.sleep(10)
    
    # Create ngrok tunnel if token is available
    if ngrok_token and ngrok_token.strip():
        try:
            print("🌐 Creating public tunnel with ngrok...")
            public_url = ngrok.connect(DASHBOARD_PORT)
            PUBLIC_URL = public_url
            
            print("🎉 Dashboard is ready!")
            print("=" * 50)
            print(f"🌐 Public URL: {public_url}")
            print(f"📱 Mobile-friendly: {public_url}")
            print("🔗 Click the link above to access your dashboard")
            print()
            print("⚠️ Security Note: This URL is public and accessible to anyone")
            print("   Consider using ngrok's paid plan for password protection")
            
        except Exception as e:
            print(f"❌ Failed to create ngrok tunnel: {e}")
            print(f"🏠 Dashboard available locally at: http://localhost:{DASHBOARD_PORT}")
            print("   (Not accessible outside Colab without tunnel)")
    else:
        print(f"🏠 Dashboard running locally at: http://localhost:{DASHBOARD_PORT}")
        print("   (Not accessible outside Colab - need ngrok token for public access)")

else:
    print("💻 Local Environment: Use the previous instructions to start the dashboard manually")
    print("    This cell is only for Google Colab automatic setup")

In [ ]:
# Create a trainer for the first dataset to inspect the setup
if trainer.datasets:
    dataset_name = list(trainer.datasets.keys())[0]
    print(f"🔧 Creating trainer for dataset: {dataset_name}")
    
    try:
        hf_trainer = trainer.create_trainer(dataset_name)
        print(f"✅ HuggingFace trainer created")
        print(f"   Training dataset: {len(hf_trainer.train_dataset):,} examples")
        if hf_trainer.eval_dataset:
            print(f"   Eval dataset: {len(hf_trainer.eval_dataset):,} examples")
        
        # Show training arguments
        args = hf_trainer.args
        print(f"\n⚙️  Training arguments:")
        print(f"   Output dir: {args.output_dir}")
        print(f"   Learning rate: {args.learning_rate}")
        print(f"   Batch size: {args.per_device_train_batch_size}")
        print(f"   Max steps: {args.max_steps}")
        print(f"   Save steps: {args.save_steps}")
        print(f"   Eval steps: {args.eval_steps}")
        print(f"   FP16: {args.fp16}")
        
    except Exception as e:
        print(f"❌ Trainer creation failed: {e}")
        import traceback
        traceback.print_exc()

## Step 6: Test Adaptive Context System

Before training, let's test the adaptive context system with different types of inputs:

In [ ]:
# Test the adaptive context system
print("🧠 Testing Adaptive Context System")
print("=" * 50)

# Test different types of inputs
test_inputs = {
    "simple_chat": "Hello! How are you today?",
    
    "code_task": """
def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)

# This is a recursive implementation
# Could be optimized with dynamic programming
for i in range(10):
    print(f"fib({i}) = {fibonacci(i)}")
""",
    
    "reasoning_task": """
Let me think through this step by step. If we have a logical puzzle where:
1. All cats are animals
2. Some animals are pets  
3. No pets are wild
4. Some cats are wild

We need to determine if there's a contradiction. Let me analyze each statement carefully
and see if they can all be true simultaneously. This requires careful logical reasoning
to avoid making invalid inferences.
""",
    
    "long_document": """
This is a comprehensive research paper on machine learning that covers multiple aspects
of the field. The introduction provides background on artificial intelligence and its
historical development. The methodology section describes various approaches including
supervised learning, unsupervised learning, and reinforcement learning paradigms.
""" + " The paper continues with detailed analysis." * 50  # Make it longer
}

# Test each input type
for task_type, text in test_inputs.items():
    print(f"\n🔍 Testing: {task_type}")
    print(f"Input length: {len(text)} characters")
    
    # Tokenize the input
    inputs = trainer.tokenizer(text, return_tensors="pt", truncation=False)
    input_ids = inputs["input_ids"]
    token_count = input_ids.shape[1]
    print(f"Token count: {token_count}")
    
    # Get current context info
    initial_context = trainer.model.get_context_info()['current_context_length']
    
    # Test the model (this should trigger adaptive context)
    trainer.model.eval()
    with torch.no_grad():
        try:
            # This forward pass will trigger context adaptation
            outputs = trainer.model(input_ids, return_dict=True)
            
            # Check if context adapted
            final_context = trainer.model.get_context_info()['current_context_length']
            
            print(f"Context: {initial_context:,} → {final_context:,} tokens")
            if final_context != initial_context:
                print(f"✅ Context adapted for {task_type}")
            else:
                print(f"→ Context unchanged for {task_type}")
                
        except Exception as e:
            print(f"❌ Error processing {task_type}: {e}")

## Step 7: Run Training

Now let's run the actual training! We'll train for a short period to demonstrate the system:

In [ ]:
# Warning: This will actually train the model!
print("⚠️  TRAINING WARNING")
print("=" * 50)
print("The next cell will run actual training.")
print("This may take several minutes and will:")
print("• Download datasets from HuggingFace")
print("• Train the model for 200 steps")
print("• Show parameter growth during training")
print("• Save model checkpoints")
print("")
print("Set RUN_TRAINING = True to proceed")

RUN_TRAINING = False  # Set to True to actually run training

if RUN_TRAINING:
    print("🚀 Starting training pipeline...")
else:
    print("🛑 Training skipped (set RUN_TRAINING = True to run)")

In [ ]:
# Run training if enabled
if RUN_TRAINING:
    print("🚀 Starting Arbor YAML Training Pipeline with Live Dashboard")
    print("=" * 60)
    
    try:
        # Start monitoring
        if 'training_monitor' in locals():
            training_monitor.start_monitoring()
            print("📊 Training monitor started - metrics will be saved for dashboard")
        
        # Show dashboard access info
        if IN_COLAB and 'PUBLIC_URL' in globals() and PUBLIC_URL:
            print(f"🌐 Monitor training at: {PUBLIC_URL}")
        elif not IN_COLAB:
            dashboard_port = tutorial_config['logging']['arbor_tracking']['dashboard_port']
            print(f"🌐 Monitor training at: http://localhost:{dashboard_port}")
        else:
            print("📊 Dashboard metrics being collected (check dashboard startup cells for URL)")
        
        # This runs the complete training pipeline
        trainer.train()
        
        print("\n🎉 Training completed successfully!")
        
        # Show final model stats
        final_params = trainer.model.param_count()
        print(f"📊 Final model size: {final_params:,} parameters")
        
        # Show training outputs
        output_dir = Path(trainer.config.training_config['output_dir'])
        if output_dir.exists():
            saved_models = list(output_dir.glob("*/"))
            print(f"💾 Saved {len(saved_models)} model checkpoints:")
            for model_dir in saved_models:
                print(f"   📁 {model_dir.name}")
        
        # Stop monitoring and generate report
        if 'training_monitor' in locals():
            training_monitor.stop_monitoring()
            report_file = training_monitor.export_training_report()
            print(f"📄 Training report saved: {report_file}")
            
            # Show final dashboard link
            if IN_COLAB and 'PUBLIC_URL' in globals() and PUBLIC_URL:
                print(f"\n🌐 View final metrics at: {PUBLIC_URL}")
            elif not IN_COLAB:
                dashboard_port = tutorial_config['logging']['arbor_tracking']['dashboard_port']
                print(f"\n🌐 View final metrics at: http://localhost:{dashboard_port}")
        
    except Exception as e:
        print(f"❌ Training failed: {e}")
        import traceback
        traceback.print_exc()
        
        # Stop monitoring on error
        if 'training_monitor' in locals():
            training_monitor.stop_monitoring()
        
else:
    # Simulate what training would show with Arbor tracking
    print("📋 Training simulation (would show):")
    print("🌱 Initialized Arbor trainer with config: configs/tutorial_config.yaml")
    print("📊 Arbor tracking system initialized:")
    
    if IN_COLAB:
        print("   • Dashboard ready with public ngrok URL")
        if 'PUBLIC_URL' in globals() and PUBLIC_URL:
            print(f"   • Access at: {PUBLIC_URL}")
        else:
            print("   • Public URL will be generated when dashboard starts")
    else:
        print("   • Dashboard ready at http://localhost:8501")
    
    print("   • Metrics saved to: ./training_logs")
    print("   • Real-time monitoring enabled")
    print("   • Alert system active")
    print(" Downloading fresh Hermes-4-405B tokenizer...")
    print("✅ Successfully loaded fresh Hermes-4-405B tokenizer")
    print("✅ Created Arbor model: 347,394,048 parameters")
    print("🧠 Adaptive context enabled:")
    print("   Range: 512 - 32,768")
    print("   Supported tasks: 4")
    print("🌱 Growth monitoring enabled:")
    print("   Factor: 1.5x")
    print("   Max steps: 4")
    print("📚 Loading datasets...")
    print("   ✅ tiny_stories: 1,000 examples")
    print("   ✅ code_samples: 500 examples")
    print("")
    print("🎯 Training on tiny_stories...")
    print("   📊 Dashboard: Real-time loss curves, layer utilization heatmaps")
    print("   📊 Parameters: 347,394,048 → 347,894,048 (growth occurred)")
    print("   🔔 Alert: Layer growth event detected")
    print("   ✅ tiny_stories complete!")
    print("")
    print("🎯 Training on code_samples...")
    print("   📊 Dashboard: Architecture visualization updated")
    print("   📊 Parameters: 347,894,048 → 348,394,048 (growth occurred)")
    print("   🔔 Alert: Performance threshold reached")
    print("   ✅ code_samples complete!")
    print("")
    print("🎉 Training pipeline complete!")
    
    if IN_COLAB:
        print("🌐 View comprehensive analytics at your public dashboard URL")
    else:
        print("📊 Final dashboard shows comprehensive training analytics")
    
    print("📄 Training report exported with growth timeline")

## Step 8: Test the Trained Model

Let's test our model (or demonstrate what testing would look like) with different task types to see how the adaptive context system works:

In [ ]:
# Test model generation with different tasks
print("🧪 Testing Trained Model")
print("=" * 50)

test_prompts = {
    "story": "Once upon a time, in a magical forest",
    "code": "# Python function to calculate factorial\ndef factorial(n):",
    "reasoning": "Let me solve this step by step. The problem is:",
    "chat": "User: What's the weather like today?\nAssistant:"
}

for task_type, prompt in test_prompts.items():
    print(f"\n🎯 Testing {task_type} task:")
    print(f"Prompt: {prompt}")
    
    # Tokenize prompt
    inputs = trainer.tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"]
    
    # Show what would happen with adaptive context
    print(f"Input tokens: {input_ids.shape[1]}")
    
    if RUN_TRAINING:
        # Actually test the trained model
        trainer.model.eval()
        with torch.no_grad():
            try:
                # Generate response
                generated = trainer.model.generate(
                    input_ids,
                    max_new_tokens=50,
                    temperature=0.7,
                    do_sample=True
                )
                
                # Decode response
                response = trainer.tokenizer.decode(generated[0], skip_special_tokens=True)
                print(f"Generated: {response[len(prompt):]}")
                
                # Show context info
                context_info = trainer.model.get_context_info()
                print(f"Context used: {context_info['current_context_length']:,} tokens")
                
            except Exception as e:
                print(f"❌ Generation failed: {e}")
    else:
        # Simulate what would happen
        simulated_contexts = {"story": 2048, "code": 4096, "reasoning": 8192, "chat": 1024}
        print(f"Would adapt context to: {simulated_contexts[task_type]:,} tokens")
        print(f"Would generate appropriate {task_type} response")

## Step 9: Configuration Tips and Best Practices

Here are some tips for customizing your YAML training configurations:

In [ ]:
# Configuration tips and best practices
print("💡 YAML Configuration Tips")
print("=" * 50)

tips = {
    "Model Size": {
        "Small (100M)": "hidden_size: 512, num_layers: 12",
        "Medium (500M)": "hidden_size: 1024, num_layers: 24", 
        "Large (1B)": "hidden_size: 1536, num_layers: 32"
    },
    
    "Context Lengths": {
        "Short tasks": "max 4K tokens (chat, Q&A)",
        "Medium tasks": "4K-16K tokens (code, creative)",
        "Long tasks": "16K+ tokens (documents, reasoning)"
    },
    
    "Growth Settings": {
        "Conservative": "factor: 1.25, threshold: 0.95",
        "Moderate": "factor: 1.5, threshold: 0.9",
        "Aggressive": "factor: 2.0, threshold: 0.85"
    },
    
    "Training Speed": {
        "Fast prototyping": "small datasets, few steps",
        "Full training": "complete datasets, many steps", 
        "Production": "multiple epochs, careful validation"
    }
}

for category, options in tips.items():
    print(f"\n🔧 {category}:")
    for option, description in options.items():
        print(f"   {option}: {description}")

print(f"\n📋 Common YAML patterns:")
print("""
# Full monitoring setup for research
adaptive_context: 
  enabled: true
growth:
  enabled: true
logging:
  arbor_tracking:
    enabled: true
    save_dir: './training_logs'
    dashboard_port: 8501
    alerts:
      enabled: true

# Minimal setup for testing  
adaptive_context:
  enabled: false
growth:
  enabled: false
logging:
  arbor_tracking:
    enabled: false
  console:
    enabled: true
datasets:
  - name: "test"
    source: "roneneldan/TinyStories"
    split: "train[:100]"

# Production setup with full tracking
logging:
  arbor_tracking:
    enabled: true
    save_dir: './production_logs'
    update_interval: 0.5
    dashboard_port: 8501
    alerts:
      enabled: true
      email_notifications: true
      webhook_url: "https://your-webhook-url.com"
""")

## Summary

Congratulations! You've learned how to use the Arbor YAML training system with real-time monitoring. Here's what we covered:

### ✅ **What You Learned:**

1. **📋 YAML Configuration** - How to create and customize training configs
2. **🧠 Adaptive Context** - Task-aware context window adaptation  
3. **🌱 Dynamic Growth** - Parameter expansion during training
4. **🚀 Easy Training** - One-command training with `python train.py config.yaml`
5. **📊 Live Monitoring** - Real-time dashboard with training visualization
6. **🧪 Testing & Validation** - How to test trained models

### 🎯 **Key Benefits:**

- **Simple**: Just edit YAML, no complex code
- **Powerful**: Full control over model architecture and training
- **Smart**: Automatic context adaptation and parameter growth
- **Visual**: Real-time dashboard with live metrics and architecture visualization
- **Production Ready**: HuggingFace integration and comprehensive monitoring

### 🚀 **Next Steps:**

1. **Customize** your own YAML config for your use case
2. **Train** with real datasets for your domain
3. **Monitor** training with the Arbor dashboard at http://localhost:8501
4. **Deploy** trained models to HuggingFace Hub

### 📊 **Dashboard Features:**

- **Live Metrics**: Training loss, learning rate, gradient norms in real-time
- **Architecture View**: Interactive model visualization with layer utilization
- **Growth Tracking**: Parameter and layer growth timeline
- **Alert System**: Automatic notifications for training events
- **Analytics**: Performance statistics and comprehensive reports

The YAML trainer with Arbor dashboard makes it incredibly easy to experiment with cutting-edge transformer architectures while monitoring everything in real-time!

## 🌐 Google Colab Users - Public Dashboard Access

### 🎯 **For Google Colab Users:**

Since Google Colab doesn't allow direct access to `localhost`, we use **ngrok** to create a public tunnel to your dashboard.

#### 🚀 **Quick Setup:**

1. **Get ngrok token** (free): Go to [ngrok.com](https://ngrok.com) → Sign up → Get your auth token
2. **Run the environment setup cells** above to detect Colab and install packages
3. **Run the dashboard startup cell** - enter your ngrok token when prompted
4. **Get your public URL** - a unique URL like `https://abc123.ngrok.io`
5. **Monitor training** - click the public URL to access your dashboard from anywhere!

#### 📊 **Dashboard Features in Colab:**

- **📱 Mobile-Responsive**: Perfect for monitoring on your phone
- **🌐 Public Access**: Share the URL with team members
- **🔄 Real-Time Updates**: Just like local development
- **📈 Full Analytics**: All features work identically

#### ⚠️ **Important Notes:**

- **Public URL**: Free ngrok URLs are accessible by anyone with the link
- **Session Timeout**: URLs expire when you restart the Colab session
- **Data Privacy**: Consider ngrok Pro for password protection in production

#### 💡 **Pro Tips:**

- Keep this notebook tab open while training runs
- Bookmark the ngrok URL for easy access
- Save important results to Google Drive
- Consider upgrading to ngrok Pro for private URLs

### 🔗 **URL Structure:**
```
Local: http://localhost:8501
Colab: https://[random].ngrok.io
```

Both URLs provide identical dashboard functionality!

In [ ]:
# Cleanup and final info
print("🧹 Cleanup and Final Info")
print("=" * 50)

# Show created files
created_files = [
    "configs/tutorial_config.yaml",
    "training_logs/" if 'training_monitor' in locals() else "training_logs/ (would be created)",
    "tutorial_output/" if RUN_TRAINING else "tutorial_output/ (would be created)"
]

print("📁 Files created during this tutorial:")
for file in created_files:
    if Path(file).exists() or "would be" in file:
        print(f"   ✅ {file}")

# Environment-specific instructions
if IN_COLAB:
    print(f"\n🌐 Google Colab Instructions:")
    print(f"=" * 30)
    
    print(f"🎯 To run training yourself:")
    print(f"   1. Set RUN_TRAINING = True in the training execution cell")
    print(f"   2. Make sure you've run the dashboard startup cells above")
    print(f"   3. Use the public ngrok URL to monitor training")
    
    print(f"\n🌐 Dashboard Access:")
    if 'PUBLIC_URL' in globals() and PUBLIC_URL:
        print(f"   🔗 Current public URL: {PUBLIC_URL}")
        print(f"   📱 Mobile-friendly and shareable")
    else:
        print(f"   🚀 Run the 'Start Dashboard in Colab' cell to get public URL")
        print(f"   🔑 Remember to set your ngrok auth token")
    
    print(f"\n⚠️ Colab Limitations:")
    print(f"   • Dashboard URL changes each time you restart")
    print(f"   • Free ngrok URLs are public (anyone with link can access)")
    print(f"   • Session will timeout if inactive for too long")
    
    print(f"\n💡 Tips for Colab:")
    print(f"   • Keep this notebook tab open while training")
    print(f"   • Save important results to Google Drive")
    print(f"   • Consider ngrok Pro for password-protected URLs")

else:
    print(f"\n💻 Local Development Instructions:")
    print(f"=" * 35)
    
    dashboard_port = tutorial_config['logging']['arbor_tracking']['dashboard_port']
    
    print(f"🎯 To run training yourself:")
    print(f"   1. Set RUN_TRAINING = True in cell 23")
    print(f"   2. Or run: python train.py configs/tutorial_config.yaml")

    print(f"\n🌐 Dashboard Access (Local):")
    print(f"   1. Start dashboard: streamlit run arbor/tracking/dashboard.py --server.port={dashboard_port}")
    print(f"   2. Open browser: http://localhost:{dashboard_port}")
    print(f"   3. Or use launcher: python launch_training_dashboard.py")

print(f"\n📊 Dashboard Features (All Environments):")
print(f"   • 📈 Live training metrics with real-time updates")
print(f"   • 🏗️ Interactive model architecture visualization")
print(f"   • 🌱 Parameter and layer growth tracking")
print(f"   • 🔔 Alert system with configurable notifications")
print(f"   • 📊 Performance analytics and export capabilities")
print(f"   • 📱 Mobile-responsive design (great for Colab!)")

print(f"\n🔧 To customize:")
print(f"   1. Edit configs/tutorial_config.yaml")
print(f"   2. Adjust model size, datasets, training steps")
print(f"   3. Enable/disable adaptive context and growth")
print(f"   4. Configure dashboard alerts and monitoring")

print(f"\n📚 For more examples:")
print(f"   • Check configs/example_config.yaml")
if not IN_COLAB:
    print(f"   • Run python examples/training_with_dashboard.py")
    print(f"   • See COMPLETE_USAGE_GUIDE.md for full documentation")
else:
    print(f"   • Upload examples to Colab for local exploration")
    print(f"   • Check the GitHub repo for complete documentation")

print(f"\n🌱 Happy training with Arbor and live monitoring!")
if IN_COLAB:
    print(f"🌐 Perfect for Google Colab with public dashboard access!")
else:
    print(f"🚀 The future of adaptive AI training is here!")

# Show ngrok tunnel info if in Colab
if IN_COLAB and 'PUBLIC_URL' in globals() and PUBLIC_URL:
    print(f"\n🔗 Current Dashboard: {PUBLIC_URL}")
    print(f"💾 Bookmark this URL to monitor your training!")